## Script to get NLTE corrections from inspect-stars.com
Note: This notebook isn't all-encompassing, and must have the site functioning in order for the script to work.

This script uses data from:

**Lithium**:
- Lind, Asplund, & Barklem, 2009, A&A, 503, 541

**Oxygen**: 
- Amarsi, Asplund, Collet & Leenaarts 2015, MNRAS, 454, 11 

**Sodium**: 
- Lind, Asplund, Barklem, & Belyaev, 2011, A&A, 528, 103

**Magnesium**: 
- Osorio, Barklem, Lind, et al., 2015, A&A, 579, 530
- Osorio & Barklem 2015, arXiv:1510.05165

**Titanium**: 
- Bergemann 2011, MNRAS, 413, 2184

**Iron**: 
- Bergemann, Lind, Collet, Asplund, & Magic, 2012, MNRAS
- Lind, Bergemann, & Asplund, 2012, MNRAS

**Strontium**: 
- Bergemann, Hansen, Bautista & Ruchti 2012, A&A, 546, 90

In [1]:
import numpy as np
import pandas as pd
import re
from urllib.request import urlopen

In [2]:
app_url = 'http://www.inspect-stars.com/cp/application.py/'
elem_list = ['Li', 'O', 'Na', 'Mg', 'Ti', 'Fe', 'Sr']

In [3]:
# Returns the index for the line in the dict, or if it is within dw, returns the
#  nearest line (index).
def check_near(line, lines_dict):
    # TOLERANCE, change if you wish.
    dw = 0.2
    
    keys = lines_dict.keys()
    values = list(lines_dict.values())
    
    if line in keys:
        return lines_dict[line]
    else:
        float_keys = [float(i) for i in keys]
        delta_float_keys = np.abs(np.array(float_keys) - float(line))
        min_ind = delta_float_keys.argmin()
        
        if delta_float_keys[min_ind] <= dw:
            return values[min_ind]
        else:
            return None

In [4]:
# Helper function to check that input parameters are within bounds
def find_bounds_html(html, elem, n):
    pattern = 'class=""></td><td>.*?</td></tr>'

    lst = []
    
    for m in re.finditer(pattern, html):
        trunc = html[m.start():m.end()]
        trunc = trunc.strip('class=""></td></tr>[]').split(',')
        i = [float(j) for j in trunc]
        lst.append(i)
        
    return(lst)

In [5]:
def return_params(elem, e, t, g, f, v, line, EW = False):
    """
    DESCRIPTION -------------------------------------------------------------
        Given input parameters, return the NLTE correction performed by INSPECT.
    
    PARAMETERS --------------------------------------------------------------
        elem (str) - a chemical element that is in the format of the contents of
            elem_list, defined in the second code-block.
        e (float) - if EW == True: the equivalent width of element at line
                    if EW == False: the abundance of element at line
        t (float) - the effective temperature of your star
        g (float) - the surface gravity of your star
        f (float) - the metallicity [Fe/H] of your star
        v (float) - the micro-turbulent velocity of your star
        line (float) - the line at which you want the NLTE correction to be performed
        
    RETURNS ----------------------------------------------------------------
        list(float) - the returned NLTE correction at element X in the format:
            [elem,
            line,
            EW (mA), 
            A(X) LTE, 
            A(X) NLTE, 
            Delta, 
            [X/Fe] NLTE]
            
            If the correction is not able to be performed, each value is replaced
            with '888'
    """
    # If the calculation can not be performed for any reason, this will be
    #  returned
    issue_response = [elem, line, 999, 999, 999, 999, 999]
    issue_txt = ':' + str(elem) + ' @ ' + str(line) + ' '
    
    input_nums = [e, t, g, f, v]
    input_names = ['Abundance A(X)', 'Temp', 'Log(g)', 'Metallicity', 'Microturbulence']
    if EW == True:
        input_names[0] = 'EW'
    
    # Checking if the inputted element is able to be processed
    if elem in elem_list:
        elem_url = app_url + 'nonlte_from_lte?element_name=' + elem
        if EW == True:
            elem_url = app_url + 'A_from_e?element_name=' + elem
    else:
        print('Sorry, element not in element list' + issue_txt)
        return issue_response
    
    # Because Ti & Fe don't take the metallicity input, this is necessary
    n = 5
    if elem in ['Ti', 'Fe']:
        input_nums.pop(3)
        input_names.pop(3)
        n = 4
    
    # Creating a list with all of the wavelengths available for this element
    lines_dict = {}
    
    html = urlopen(elem_url).read().decode("utf-8")
    pattern = "<option.*?>.*?</option.*?>"
    for match in re.finditer(pattern, html):
        trunc = html[match.start():match.end()]
        trunc = trunc.strip('<option value"></=').replace('"', '').split(">")
        if len(trunc) > 1:
            lines_dict[trunc[1]] = int(trunc[0])
    
    # Checking all of the bounds for the input-able parameters
    bounds = find_bounds_html(html, elem, n)
    for ind, input_num in enumerate(input_nums):
        bound = bounds[ind]
        if not (input_num >= bound[0]) & (input_num <= bound[1]):
            print('Sorry, cannot perform calculation' + issue_txt)
            print(input_names[ind], 'must be within', bound)
            return issue_response
    
    line_index = check_near(line, lines_dict)
    if line_index == None:
        print('Sorry, input wavelength not in list' + issue_txt)
        return issue_response

    # Actually submitting the inputs to the NLTE correction
    # Note, for Ti & Fe adding the metallicity doesn't affect the product, so no
    #  need to actually go and remove it from the submission
    
    url_extension = '&A_lte={}&t={}&g={}&f={}&x={}&wi={}'.format(e, t, g, f, v, line_index)
    if EW == True:
        url_extension = '&e={}&t={}&g={}&f={}&x={}&wi={}'.format(e, t, g, f, v, line_index)
    
    submit_url = elem_url + url_extension
    
    submit_html = urlopen(submit_url).read().decode("utf-8")
    
    if 'Calculation failed' in submit_html:
        print('No data for this equivalent width' + issue_txt)
        return issue_response
    
    results = submit_html.split('pre')[1].split('\n')[3].split('\t')
    
    # For results that can't be computed, replace nan w/ 999 just for easier handling
    #  and turn all strings into floats at the same time
    for count, i in enumerate(results):
        if i == 'nan':
            results[count] = 999
        else: 
            results[count] = eval(i)

    results.insert(0, elem)
    results.insert(1, line)
    
    return results

## Example of one line usage

In [6]:
return_params('Ok', 20, 6000, 2, -3, 1, 6103.6, EW=True)

Sorry, element not in element list:Ok @ 6103.6 


['Ok', 6103.6, 999, 999, 999, 999, 999]

In [7]:
return_params('Li', 4.2, 5000, 3.24, -3, 1.5, 6103.6)

['Li', 6103.6, 182.67, 4.2, 4.126, -0.074, 6.076]

### For easy use, input data format should follow a convention similar to:

```python
df = pd.DataFrame(data_array)
df.columns = ['elem', 'EW', 'line'] # or ['elem', 'A(X)', 'line']
star_info = ['T', 'log(g)', '[Fe/H]', 'vt'] # Can import directly from the atmosphere params csv
```

#### Note: the element name column ('elem' in the example) should follow the convention in elem_list.
To quickly convert a numeric id to the string required to run the script, using pandas,
I find that this following method works great:

```python
elem_dict = {3:'Li', 8:'O', 11:'Na', 12:'Mg', 22:'Ti', 26:'Fe', 38:'Sr'}
df[['elem']] = df[['elem']].round().replace(elem_dict)
```

#### And this is a quick way to run multiple lines into the script
```python
data = []
for i in range(len(df['A(X)'])):
    A = df['A(X)'][i]
    elem = df['elem'][i]
    line = df['line'][i]
    params_i = return_params(elem, A, *star_info, line) # Runs one line at a time
    data.append(params_i)

data = np.vstack(data)

df = pd.DataFrame(data)
elem = 'X'
sub = 'Fe'
if elem == 'Fe':
    sub = 'H'

header = 'Elem:Line:EW [mA]:A({}) LTE:A({}) NLTE:Delta:[{}/{}]'.format(elem, elem, elem, sub).split(':')
df.columns= header
```

## Example of multi-line usage:

In [13]:
elem = [3, 12, 11, 26.1, 9, 5, 22, 22.1]
ax = [3.3, 4, 4, 5.7, 5, 8, 3.8, 4]
line = [6103.6, 5172.68, 5688.2, 3815.84, 5638.2, 8392.9, 5702.67, 6092.798]

df = pd.DataFrame(np.transpose([elem, ax, line]))
df.columns = ['elem', 'A(X)', 'line']
star_info = [6400, 3, -3, 1.5]

elem_dict = {3:'Li', 8:'O', 11:'Na', 12:'Mg', 22:'Ti', 26:'Fe', 38:'Sr'}
df[['elem']] = df[['elem']].round().replace(elem_dict)

In [14]:
data = []
for i in range(len(df['A(X)'])):
    A = df['A(X)'][i]
    elem = df['elem'][i]
    line = df['line'][i]
    params_i = return_params(elem, A, *star_info, line) # Runs one line at a time
    data.append(params_i)

data = np.vstack(data)

results = pd.DataFrame(data)
elem = 'X'
sub = 'Fe'
if elem == 'Fe':
    sub = 'H'

header = 'Elem:Line:EW [mA]:A({}) LTE:A({}) NLTE:Delta:[{}/{}]'.format(elem, elem, elem, sub).split(':')
results.columns= header

No data for this equivalent width:Mg @ 5172.68 
Sorry, element not in element list:9.0 @ 5638.2 
Sorry, element not in element list:5.0 @ 8392.9 
Sorry, cannot perform calculation:Ti @ 5702.67 
Microturbulence must be within [1.0, 1.0]
Sorry, cannot perform calculation:Ti @ 6092.798 
Microturbulence must be within [1.0, 1.0]


In [15]:
print(results)

  Elem      Line EW [mA] A(X) LTE A(X) NLTE   Delta [X/Fe]
0   Li    6103.6   13.71      3.3     3.363   0.063  5.313
1   Mg   5172.68     999      999       999     999    999
2   Na    5688.2    2.58      4.0     3.911  -0.089  0.661
3   Fe   3815.84  122.82      5.7      5.79    0.09  -1.66
4  9.0    5638.2   999.0    999.0     999.0   999.0  999.0
5  5.0    8392.9   999.0    999.0     999.0   999.0  999.0
6   Ti   5702.67     999      999       999     999    999
7   Ti  6092.798     999      999       999     999    999
